**Foreword:**

This notebook is part of a workspace that demonstrates some approaches for working with [gnomAD data](https://gnomad.broadinstitute.org/downloads/) in the cloud. In Terra, this notebook runs out of the box on the default Hail environment. If you wish to use this notebook in a different environment, you may need to install relevant package dependencies manually.


## Initial setup

### Check environment requirements

**IMPORTANT!** This notebook has special environment requirements. When running in Terra, makes sure to select the Hail environment under the Application configuration section in the Cloud Enviroment menu.

### Load dependencies

In [1]:
# Load libraries

# General use and data science packages
import os
import io
import json
import pandas as pd

# Hail-specific packages
import hail as hl

# FISS: A library of functions for querying the Terra API 
# (FireCloud was the name of an earlier version of Terra)
from firecloud import fiss

### Start a Hail session

In [2]:
# Initialize the Hail engine
hl.init(default_reference = "GRCh38", log = 'gnomAD-with-Hail.log')

Running on Apache Spark version 2.4.4
SparkUI available at http://saturn-bb6f99f9-fbbd-4b1b-825c-b64098814bab-m.c.terra-outreach.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.57-582b2e31b8bd
LOGGING: writing to gnomAD-with-Hail.log


## Option A: Working with the pre-built Hail tables

As part of the gnomAD v3.1 release, the gnomAD team generated Hail tables for both the sites-only version of the dataset and the subset with sample-level genotypes. Let's have a look at how we can load those into our notebook for analysis. 

### Look up the Hail table files in the Workspace Data table

We could look up the Hail table file locations on the gnomAD website, but as you may recall, we already set up a list of those files in the Workspace Data table. So now, we can easily look up the file paths there. 

In [3]:
# Store the paths in some handy variables
sites_only_ht = "gs://gnomad-public-requester-pays/release/3.1/ht/genomes/gnomad.genomes.v3.1.sites.ht"
hgdp_1kg_subset_mt = "gs://gnomad-public-requester-pays/release/3.1/mt/genomes/gnomad.genomes.v3.1.hgdp_1kg_subset_dense.mt"

### Import the sites-only table 

Since Hail can run directly on data stored in GCS, we can read in the sites-only table, which is provided as a Hail Table (`ht`), with a single command (`read_table()`) and be ready to run some simple operations right away.

In [4]:
# Read in the table
sites_only_table = hl.read_table(sites_only_ht)

### Run some basic operations with Hail

Next we'll use the `describe()` function to get a summary of the table, and `count()` to count variant records.

In [5]:
# Get a summary of the table structure
sites_only_table.describe()

----------------------------------------
Global fields:
    'freq_meta': array<dict<str, str>> 
    'freq_index_dict': dict<str, int32> 
    'faf_index_dict': dict<str, int32> 
    'faf_meta': array<dict<str, str>> 
    'vep_version': str 
    'vep_csq_header': str 
    'dbsnp_version': str 
    'filtering_model': struct {
        model_name: str, 
        score_name: str, 
        snv_cutoff: struct {
            bin: float64, 
            min_score: float64
        }, 
        indel_cutoff: struct {
            bin: float64, 
            min_score: float64
        }, 
        model_id: str, 
        snv_training_variables: array<str>, 
        indel_training_variables: array<str>
    } 
    'age_distribution': struct {
        bin_edges: array<float64>, 
        bin_freq: array<int32>, 
        n_smaller: int32, 
        n_larger: int32
    } 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'freq': array<struct {
       

In [6]:
# Count variant record
sites_only_table.count()

759302267

This output means that the dataset contains 759,302,267 variant records.

### Import the gnomAD subset with genotypes

Now let's try loading in the gnomAD subset with genotypes, which is provided as a MatrixTable -- a somewhat more complex version of the Table format, with its own read function, `read_matrix_table()`.

In [7]:
# Read in the table
hgdp_1kg_subset_matrix = hl.read_matrix_table(hgdp_1kg_subset_mt)

In [8]:
# Get a summary of the table structure
hgdp_1kg_subset_matrix.describe()

----------------------------------------
Global fields:
    'global_annotation_descriptions': struct {
        sex_imputation_ploidy_cutoffs: struct {
            Description: str
        }, 
        population_inference_pca_metrics: struct {
            Description: str
        }, 
        hard_filter_cutoffs: struct {
            Description: str
        }, 
        cohort_freq_meta: struct {
            Description: str
        }, 
        gnomad_freq_meta: struct {
            Description: str
        }, 
        cohort_freq_index_dict: struct {
            Description: str
        }, 
        gnomad_freq_index_dict: struct {
            Description: str
        }, 
        gnomad_faf_index_dict: struct {
            Description: str
        }, 
        gnomad_faf_meta: struct {
            Description: str
        }, 
        vep_version: struct {
            Description: str
        }, 
        vep_csq_header: struct {
            Description: str
        }, 
        dbsnp_versio

In [9]:
# Count variants and samples 
hgdp_1kg_subset_matrix.count()

(175312130, 3942)

You see that this time the output shows a second number -- specifically, the number of samples included in this version of the dataset.

### Apply basic variant quality control

Hail includes some variant quality control functions, so let's try applying that to the subset data now.

In [10]:
# Run variant quality control
subset_qc = hl.variant_qc(hgdp_1kg_subset_matrix)

This produces a new version of the matrix with some additional annotations, which you can explore using the same `describe()` function as earlier if you want to investigate how it differs. You can also use `mt.rows().show()` to display a few actual records.

In [11]:
# Examine QC results
subset_qc.rows().show(5)

+---------------+----------------+----------------+------------+
| locus         | alleles        | rsid           | AS_lowqual |
+---------------+----------------+----------------+------------+
| locus<GRCh38> | array<str>     | str            |       bool |
+---------------+----------------+----------------+------------+
| chr1:10055    | ["T","C"]      | NA             |      false |
| chr1:10061    | ["T","C"]      | NA             |      false |
| chr1:10109    | ["A","T"]      | "rs376007522"  |      false |
| chr1:10109    | ["AACCCT","A"] | "rs1462685959" |      false |
| chr1:10114    | ["T","C"]      | "rs1570391787" |      false |
+---------------+----------------+----------------+------------+

+------------------------+
| telomere_or_centromere |
+------------------------+
|                   bool |
+------------------------+
|                  false |
|                  false |
|                  false |
|                  false |
|                  false |
+------------------------+

+------------------------------------------------------------------------------+
| cohort_freq                                                                  |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,3190,0),(1,2.80e-04,3574,0),(0,0.00e+00,56,0),(0,0.00e+00,54... |
| [(0,0.00e+00,3480,0),(1,2.67e-04,3740,0),(0,0.00e+00,62,0),(0,0.00e+00,48... |
| [(0,0.00e+00,346,0),(1,1.83e-03,546,0),(0,0.00e+00,12,0),(0,0.00e+00,2,0)... |
| [(8,2.30e-02,348,0),(18,3.30e-02,546,1),(0,0.00e+00,12,0),(0,0.00e+00,2,0... |
| [(0,0.00e+00,450,0),(1,1.81e-03,552,0),(0,0.00e+00,12,0),(0,0.00e+00,4,0)... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| gnomad_freq                                                                  |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(1,1.06e-05,94152,0),(5,4.65e-05,107630,0),(0,0.00e+00,44378,0),(1,1.80e... |
| [(0,0.00e+00,108768,0),(2,1.70e-05,117852,0),(0,0.00e+00,50916,0),(0,0.00... |
| [(0,0.00e+00,1556,0),(1,1.51e-04,6624,0),(0,0.00e+00,658,0),(0,0.00e+00,1... |
| [(107,6.91e-02,1548,0),(149,2.25e-02,6624,4),(57,8.74e-02,652,0),(9,5.56e... |
| [(5,2.25e-04,22208,0),(18,3.59e-04,50116,1),(3,2.48e-04,12078,0),(1,1.13e... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| gnomad_raw_qual_hists.gq_hist_all.bin_edges                                  |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| gnomad_raw_qual_hists.gq_hist_all.bin_freq                                   |
+------------------------------------------------------------------------------+
| array<int64>                                                     

This produces a table with a few rows showing sample-level information. You'll notice that many of the fields are filled with NA (for Not Applicable) but if you scroll all the way to the right you'll find the QC summary.

With all this in hand, you can use any Hail functionality you like to explore this dataset.

## Option B: Importing the VCF files into Hail
If we only had access to the VCF files from the gnomAD dataset, instead of the Hail tables lovingly prepared by the gnomAD team, we could still work with the data in Hail -- we would just have to load them into Hail ourselves. Here's one way to do that in Terra.

### List and retrieve the contents of the `chr_callset` Data table

Remember that we made a data table that lists all the VCF files and their index files, organized by chromosome, in the Data tab of the workspace. We could look up the VCF file paths in the table through the web UI, but that would be tedious and we're super lazy. So instead, we're going to retrieve them programmatically through the Terra API, which we interact with using a Python library called FISS.  

_Yes this is an excuse to show you how to use the API for stuff like this._

#### Set workspace environment variables

We're going to need these in order to access data tables in the workspace programmatically.

In [12]:
# Get the Google billing project name and workspace name
billing_project = os.environ['WORKSPACE_NAMESPACE']
workspace = os.environ['WORKSPACE_NAME']
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + billing_project)
print("Workspace: " + workspace)
print("Workspace storage bucket: " + bucket)

Billing project: terra-outreach
Workspace: DEMO-Working-with-gnomAD
Workspace storage bucket: gs://fc-15ed1113-c2db-430d-952a-af0bd1a75b31/


#### Find out what tables are present in the workspace
If we don't even want to have to look up the name of the table we want to access, we could query the Terra API to give us a list of the tables and a description of what they contain.

In [13]:
# List "entities" -- returns a JSON object listing and describing all data tables in the workspace
entities_list = fiss.fapi.list_entity_types(billing_project, workspace).text

# Display the data table descriptions (making the JSON more readable)
print(json.dumps(json.loads(entities_list),indent = 4, sort_keys=True))

{
    "chr_callset": {
        "attributeNames": [
            "hgdp_1kg_subset_vcf",
            "hgdp_1kg_subset_vcf_index",
            "intervals",
            "sites_validation",
            "sites_vcf",
            "sites_vcf_index"
        ],
        "count": 24,
        "idName": "chr_callset_id"
    }
}


#### Retrieve the contents of the chr_callset table 
The commands here a bit painful to read; what it boils down to is that we're grabbing the full contents of the table in TSV format, then transforming that into a Pandas dataframe.  

Note the `model="flexible"` argument: this is a legacy from an earlier version of the platform, which only allowed one specific data model. We have to specify this argument if we want to access any entities that are not represented in that data model. If you're not sure, just include this argument in the query -- it shouldn't hurt even if you are using the old data model. 

In [14]:
# Get chr_callset table contents
chr_callset_obj = fiss.fapi.get_entities_tsv(billing_project, workspace, "chr_callset", model="flexible").text
chr_callset_table = pd.read_csv(io.StringIO(chr_callset_obj), sep='\t')
chr_callset_table.rename(columns = {'entity:chr_callset_id':'chr_callset'}, inplace = True)

# Display the first few rows of the table
chr_callset_table.head()

,chr_callset,hgdp_1kg_subset_vcf,hgdp_1kg_subset_vcf_index,intervals,sites_validation,sites_vcf,sites_vcf_index
0,chr1,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gatk-test-data/intervals/wgs_calling_inte...,"[""gs://fc-15ed1113-c2db-430d-952a-af0bd1a75b31...",gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gcp-public-data--gnomad/release/3.1/vcf/g...
1,chr10,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gatk-test-data/intervals/wgs_calling_inte...,NaN,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gcp-public-data--gnomad/release/3.1/vcf/g...
2,chr11,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gatk-test-data/intervals/wgs_calling_inte...,NaN,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gcp-public-data--gnomad/release/3.1/vcf/g...
3,chr12,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gatk-test-data/intervals/wgs_calling_inte...,NaN,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gcp-public-data--gnomad/release/3.1/vcf/g...
4,chr13,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gatk-test-data/intervals/wgs_calling_inte...,NaN,gs://gcp-public-data--gnomad/release/3.1/vcf/g...,gs://gcp-public-data--gnomad/release/3.1/vcf/g...


As you can see this is a copy of the table from the Data section of the workspace. Since it contains all the file paths, we can now use them in notebook commands; you just need to know how to extract the paths you're interested in from the dataframe. 

#### Extract file paths from the dataframe
First, we set an index for the dataframe, then it's a matter of specifying which row and/or column we're interested in. Here we're grabbing the content of just a single cell, to get the file path for the sites-only VCF of chr22. 

In [15]:
# Set the dataframe index
chr_callset_table = chr_callset_table.set_index("chr_callset", drop = False)

In [16]:
# Identify a specific VCF file
chr22_sites_vcf = chr_callset_table.loc["chr22", "sites_vcf"]

# Check the path
print(chr22_sites_vcf)

gs://gcp-public-data--gnomad/release/3.1/vcf/genomes/gnomad.genomes.v3.1.sites.chr22.vcf.bgz


You could also retrieve multiple VCF files into a list by grabbing a range or the whole column from the dataframe; see the Pandas documentation for specifc usage instructions. 

### Load the contents of a VCF into a Hail MatrixTable (`mt`)
You could do a number of things with that VCF, but since the point of this notebook is to use Hail, let's look at how we would make the VCF contents available to Hail. 

The process is surprisingly simple; we give the GCS file path to Hail's VCF import function, and it will look up the necessary metadata to set up the structure of the matrix table (without retrieving any of the actual data). You can then use the `describe()` function to get an overview of what's in the file, or the `count()` function to count variant records. 

In [17]:
# Run the import function on the GCS file path
chr22_mt = hl.import_vcf(chr22_sites_vcf)

In [18]:
# Get a summary of the matrix structure
chr22_mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AN: int32, 
        AF: array<float64>, 
        popmax: array<str>, 
        faf95_popmax: array<float64>, 
        `AC-non_v2-XX`: array<int32>, 
        `AN-non_v2-XX`: int32, 
        `AF-non_v2-XX`: array<float64>, 
        `nhomalt-non_v2-XX`: array<int32>, 
        `AC-non_cancer-fin-XX`: array<int32>, 
        `AN-non_cancer-fin-XX`: int32, 
        `AF-non_cancer-fin-XX`: array<float64>, 
        `nhomalt-non_cancer-fin-XX`: array<int32>, 
        `AC-non_neuro-nfe`: array<int32>, 
        `AN-non_neuro-nfe`: int32, 
        `AF-non_neuro-nfe`: array<float64>, 
        `nhomalt-non_neuro-nfe`: array<int32>, 
        `AC-non_neu

In [19]:
# Optional: Get the variant counts. NOTE: This may take a long time for large files!
chr22_mt.count()

2020-11-09 22:56:38 Hail: INFO: Coerced sorted dataset


(11606640, 0)

That output means the file contains 11,606,640 variant site records and 0 samples, which makes sense since we're running on the sites-only file here. As an exercise, try running this again for the "hgdp_1kg_subset_vcf" version of the data, which includes sample genotypes, and see how the output changes.

You may notice that the queries you do on the per-chromosome VCFs take quite a bit longer than the equivalents on the entire dataset stored in Hail table format. That's because the Hail tables are structured specifically to make these kinds of queries super efficient, whereas VCF is a very inefficient format. 

## Appendix

### Getting help with Hail

We don't provide support for Hail, but the Hail team maintains extensive [documentation](https://hail.is/docs/) as well as an active [community forum](https://discuss.hail.is/) where you can report issues, ask questions and discuss applications of Hail with fellow researchers. You can also get the latest Hail updates by following [@hailgenetics](https://twitter.com/hailgenetics?lang=en) on Twitter. 


### Creating a local directory to store data

Cloud-engineered tools like GATK and Hail are able to run many commands directly on data stored in Google Cloud Storage (GCS), which is super convenient since it means you don't have to deal with copying files or paying for more storage space. However in some cases you may want to copy the files to local storage anyway -- for example if you need to run some tools that can't operate directly on data stored in GCS. 

Any data stored to a location under `/home/jupyter-user/notebooks` will be saved to a persistent disk (as explained [here](https://support.terra.bio/hc/en-us/articles/360049950131-Update-to-Jupyter-Notebook-environment-in-Terra-Persistent-Disk-storage-now-available) so it's usually a good idea to put your project data there.

In [20]:
# Set a variable for the path so you can reuse it
DATA = '/home/jupyter-user/notebooks/project-data'

# Create the local directory
#! mkdir {DATA}

# Check that it's there
! ls {DATA}/..

ls: cannot access '/home/jupyter-user/notebooks/project-data/..': No such file or directory


So here's how you would copy over a data file to the directory we just created.

In [21]:
# Store a GCS file path in a variable
test_file = 'gs://genomics-in-the-cloud/hello.txt'

# Copy the file to local storage
! gsutil cp {test_file} {DATA}

Copying gs://genomics-in-the-cloud/hello.txt...
/ [1 files][   20.0 B/   20.0 B]                                                
Operation completed over 1 objects/20.0 B.                                       


Keep in mind though that some of the files you're going to be working with are very large, so make sure you actually need a local copy of anything you're planning to copy over, and check how much space you have available in your environment. For example, some of the gnomAD files take up hundreds of Gb of space; if you want to work with local copies, you may need to increase the size of your disk. Remember that on the cloud, the game is to avoid working with local copies of large data. 

### Other basic data management operations

If you'd like to learn more about how to do other basic operations in notebooks, including transferring data to and from Google Cloud Storage with gsutil, check out the [tutorial notebook](https://app.terra.bio/#workspaces/help-gatk/Genomics-in-the-Cloud-v1/notebooks/launch/Genomics-Notebook-executed.ipynb) included in the [Genomics in the Cloud](https://app.terra.bio/#workspaces/help-gatk/Genomics-in-the-Cloud-v1) [book](oreil.ly/genomics-cloud) workspace. It contains a set of basic examples on that topic, as well as tips on combining Python and R code in the same notebook, spinning up an interactive IGV viewer and running programs like GATK. 